## Building the Boxmot Module

In [5]:
Home = '/kaggle/working/'
%cd {Home}
#!pip install ultralytics
#!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
#!pip install -e detectron2_repo

%cd {Home}
#!git clone https://github.com/KeeganFernandesWork/yolo_tracking
%cd yolo_tracking
#!pip install -r requirements.txt
#!pip install .

from IPython.display import clear_output
clear_output(wait=False)

In [6]:
%matplotlib inline
from IPython.display import display
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path
import numpy as np
import tracemalloc
import shutil
import time
import PIL
import cv2
import os
import sys

In [7]:
import cv2
import torch
import numpy as np
from torchvision import models, transforms
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn as nn

import os

%matplotlib inline
from IPython.display import display
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path
import numpy as np
import tracemalloc
import shutil
import time
import PIL
import cv2
import os
import sys

from boxmot import (OCSORT, BoTSORT, BYTETracker, DeepOCSORT, StrongSORT,create_tracker, get_tracker_config)

# Run after restart session
import os
import cv2
import time
import json
import torch
import shutil
import warnings
import detectron2
import numpy as np
import tracemalloc
from PIL import Image
import matplotlib.pyplot as plt
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import Boxes
from IPython.display import clear_output
from detectron2.utils.logger import setup_logger
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
warnings.filterwarnings("ignore", category=FutureWarning, message=".*torch.load.*weights_only=False.*")
setup_logger()

<_Logger detectron2 (DEBUG)>

# Functions

In [43]:
# Define the modified ResNet-50 for 16x16 images
class ModifiedResNet50(nn.Module):
    def __init__(self):
        super(ModifiedResNet50, self).__init__()
        # Load the pre-trained ResNet-50 model
        self.model = models.resnet50(pretrained=True)

        # Modify the initial convolution layer to a smaller kernel and stride
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

        # Remove the first max pooling layer to preserve spatial resolution
        self.model.maxpool = nn.Identity()

        # Modify the layers to preserve spatial dimensions as much as possible
        # Ensure later downsampling layers do not reduce size below 1x1
        # For example, layer1, layer2, layer3, layer4 are left as-is since they are critical to the ResNet structure
        # For very small inputs, ResNet's inherent stride and kernel sizes can manage reduction without going to zero.

        # Remove the fully connected layer to get feature maps directly
        self.model.fc = nn.Identity()

        # Add global average pooling to ensure the output is always 1x1 in spatial dimension
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        # Forward pass through the modified ResNet-50
        features = self.model(x)
        # Apply global average pooling to get a fixed-size output regardless of input size
        pooled_features = self.global_avg_pool(features)
        # Flatten the features to a 1D vector
        flattened_features = torch.flatten(pooled_features, 1)
        return flattened_features

    
# Function to extract features from images or bounding boxes with controlled input size
def extract_features_variable_size(crop_img, model, input_size):
    # Resize the crop to the controlled input size
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((input_size, input_size)),  # Resize to the specified input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    crop_transformed = transform(crop_img).unsqueeze(0)

    # Extract features using the CNN model
    with torch.no_grad():
        features = model(crop_transformed)

    return features.squeeze().numpy()


# Define a CNN model with a variable input size control
class AdaptiveCNN(nn.Module):
    def __init__(self, output_size=(1, 1)):
        super(AdaptiveCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # Conv layer
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # Downsample
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), # Conv layer
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # Downsample
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),# Conv layer
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # Downsample
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1), # Conv layer
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(output_size)  # Adaptive pooling to ensure fixed-size output
        )
        self.flatten = nn.Flatten()  # Flatten to convert to 1D feature vector

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)  # Flatten the feature map to a 1D feature vector
        return x


def extract_target_img(frame, target_bbox, show_crop_img=False):
    # Extract features for the target bounding box
    target_x1, target_y1, target_x2, target_y2 = target_bbox
    # Convert bounding box coordinates to integers
    target_x1 = int(target_x1)
    target_y1 = int(target_y1)
    target_x2 = int(target_x2)
    target_y2 = int(target_y2)
    target_crop = frame[target_y1:target_y2, target_x1:target_x2] #Corrected the order of x and y coordinates
    if show_crop_img:
        plt.figure(figsize=(8, 6))  # Set the figure size
        plt.imshow(target_crop) 

    #target_crop = resize_to_16x16(target_crop)
    #target_crop = transform(target_crop).unsqueeze(0)


    #with torch.no_grad():
      #target_features = extractor(target_crop).squeeze().numpy()

    return target_crop

# Function to compute cosine similarity
def compute_similarity(features1, features2):
    return cosine_similarity(features1.reshape(1, -1), features2.reshape(1, -1))[0][0]

# Eculidian Distance
def euclidean_distance(feature1, feature2):
    feature1 = np.array(feature1)
    feature2 = np.array(feature2)
    return np.linalg.norm(feature1 - feature2)

def cv2_imshow(img, size=(8, 6), show=True, save=''):
    if show:
        plt.figure(figsize=size)  # Set the figure size
        plt.axis('off')
        plt.imshow(img) 
        
    if os.path.isdir(save[:26]):
        plt.imsave(save, img)
        

# Function to filter bounding boxes based on feature similarity
def filter_bboxes(frame, bboxes, target_img, threshold=0.8, show_filter_img=False, show_scores=False):
    filtered_bboxes = []
    resnet_similarity_scores =[]
    sift_scores = []
    resnet_distance_scores =[]

    # Iterate through all bounding boxes
    for bbox in bboxes:
        # Extract features for the current bounding box
        x1, y1, x2, y2 = bbox
        x1 = int(x1)
        y1 = int(y1)
        x2 = int(x2)
        y2 = int(y2)
        crop = frame[y1:y2, x1:x2]
        if crop.size == 0:  # Skip empty crops
            continue

        min_input_size = crop.shape[0] if crop.shape[0] < crop.shape[1] else crop.shape[1]

        # Initialize the CNN model with adaptive pooling to a fixed output size
        model = AdaptiveCNN(output_size=(1, 1))  # Output size after pooling
        model.eval()  # Set the model to evaluation mode

        #model2 = ModifiedResNet50() 
        #model2.eval()

        target_img_copy = cv2.resize(target_img.copy(), (min_input_size, min_input_size))
        crop = cv2.resize(crop, (min_input_size, min_input_size))
        # Extract features from the bounding boxes with controlled input size
        target_features = extract_features_variable_size(target_img_copy, model, min_input_size)
        detection_features = extract_features_variable_size(crop, model, min_input_size)

        #target_features2 = extract_features_variable_size(target_img_copy, model2, min_input_size)
        #detection_features2 = extract_features_variable_size(crop, model2, min_input_size)

        # Compute similarity between the two feature vectors
        sift_scores.append(sift_feature_matching(target_img, crop))
        resnet_distance_scores.append(euclidean_distance(target_features, detection_features))
        resnet_similarity_scores.append(compute_similarity(target_features, detection_features))

    if show_filter_img:
        frame_copy = frame.copy()
        i=1
        for bbox in bboxes:
            x1, y1, x2, y2 = bbox
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)
            cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_copy, f"BBox{i}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            i+=1
    
        cv2_imshow(frame_copy) 

        
    try:
        max_index = np.argmax(sift_scores)
    except:
        try:
            max_index = np.argmin(resnet_distance_scores)
        except:
            try:
                max_index = np.argmax(resnet_similarity_scores)
            except:
                max_index = -1

    if show_scores:
        print('\n\n')
        print('Sift_scores:', sift_scores)
        print('Resnet Simlarity scores:', resnet_similarity_scores)
        print('Resnet Euclidean_scores:', resnet_distance_scores)
        print('Max Index:', max_index)


    return max_index

In [9]:
def sift_feature_matching(target_img, crop_img):
    # Load the two images
    img1 = crop_img  # Query image
    img2 = target_img  # Train image
    # Convert images to grayscale
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Initialize the SIFT detector
    sift = cv2.SIFT_create()

    # Detect SIFT keypoints and compute descriptors for both images
    keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2, None)

    # Initialize the BFMatcher
    bf = cv2.BFMatcher()

    # Use KNN matcher to find the best 2 matches for each descriptor
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    # Apply Lowe's ratio test to filter good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.8 * n.distance:  # Ratio test
            good_matches.append(m)

    #print(f"Number of Keypoints in Image 1: {len(keypoints1)}")
    #print(f"Number of Keypoints in Image 2: {len(keypoints2)}")
    #print(f"Number of Good Matches: {len(good_matches)}")
    #print(f"Matching Score: {matching_score:.4f}")
    return len(good_matches)


In [10]:
def setup_config(model, output_dir):
    cfg = get_cfg()

    cfg.merge_from_file(model_zoo.get_config_file(model))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
    #cfg.DATASETS.TRAIN = (train_data,)
    #cfg.DATASETS.TEST = (val_data,)
    cfg.TEST.EVAL_PERIOD = 500
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 16
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # Number of classes for Faster RCNN
    cfg.MODEL.RETINANET.NUM_CLASSES = 3  # Number of classes for Retina Net
    cfg.MODEL.MASK_ON = False
    cfg.MODEL.LOAD_PROPOSALS = False

    # Debug/verbose
    setup_logger()
    #logger = logging.getLogger("detectron2")
    #logger.setLevel(logging.DEBUG)
    #logger.info("Training has started.")
    #logger.debug("This is a detailed debug message.")

    # Create the output directory
    cfg.OUTPUT_DIR = "/kaggle/working/" + output_dir
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    return cfg


def draw_bounding_boxes(image, boxes, classes, gt, plot=False):
    # Draw ground truth bounding boxes
    if gt:
        color = (0, 0, 255)
    else:
        color = (0, 255, 0)

    item_no=1
    for box,clas in zip(boxes, classes):
        x_min, y_min, x_max, y_max = map(int, box)
        label = f'{item_no}__{clas}'
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 1)  # Blue for ground truth
        cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        item_no +=1

    if(plot==True):
        plt.figure(figsize=(12, 8))
        plt.imshow(image)
        plt.title("Predicted Bounding Boxes and Classes")
        plt.axis('off')  # Hide axis
        plt.show()

    return image


def xywhn_to_xyxy(cx, cy, w, h, image_width, image_height):
    """
    Convert YOLO format [cx, cy, w, h] with normalized values to [x_min, y_min, x_max, y_max] in pixel values.

    Parameters:
    - cx, cy: Center coordinates (normalized)
    - w, h: Width and height (normalized)
    - image_width: Width of the image in pixels
    - image_height: Height of the image in pixels

    Returns:
    - A list [x_min, y_min, x_max, y_max] in pixel values
    """
    x_min = (cx - w / 2) * image_width
    y_min = (cy - h / 2) * image_height
    x_max = (cx + w / 2) * image_width
    y_max = (cy + h / 2) * image_height
    return [x_min, y_min, x_max, y_max]

## Video Writer

In [11]:
def create_video_writer(video_cap, output_filename):

    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    writer = cv2.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    return writer

In [12]:
Home = '/kaggle/working/'
color = (0, 255, 0)  # BGR
thickness = 2
fontscale = 1

device = "cuda:0" # cuda:0 , cpu
fp16 = True # True if gpu available

out_dir = f'{Home}/Output/'
os.makedirs(out_dir, exist_ok=True)

source = '/kaggle/input/test-video-15fps/Test4_15fps.mp4'
target_label = '/kaggle/input/test-dataset/test/labels/frame_0001.txt'
first_frame = '/kaggle/input/test-dataset/test/images/frame_0001.jpg'
wgts_dir = '/kaggle/input/weight/'

model_names = ['faster_rcnn', 'retinanet', 'cascade_rcnn', 'yolo']
all_wgts = ['faster_rcnn.pth', 'retinanet.pth', 'cascade_rcnn.pth', '/kaggle/input/weight/yolo.pt']
detectron_models = ["COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml", "COCO-Detection/retinanet_R_50_FPN_1x.yaml", "Misc/cascade_mask_rcnn_R_50_FPN_1x.yaml"]

In [13]:
def load_yolo(image, weight, threshold):
    start_time = time.time()
    tracemalloc.start()
    #-----------------------
    model = YOLO(weight)
    results = model.predict(image, conf=threshold)
    pred_boxes = results[0].boxes.xyxy.cpu().numpy()  # Predicted bounding boxes
    pred_classes = results[0].boxes.data[:, 5].cpu().numpy()  # Predicted class IDs
    pred_scores = results[0].boxes.data[:, 4].cpu().numpy()  # Confidence scores
    #------------------------
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    end_time = time.time()
    execution_time = end_time - start_time
    execution_time = np.round(execution_time,2)
    return pred_boxes, pred_classes, pred_scores, execution_time


def load_detectron2(image, yaml_model, threshold, all_models=detectron_models, all_wgts=all_wgts, wgts_dir = wgts_dir):
    start_time = time.time()
    tracemalloc.start()
    #------------------------------------
    model_index = all_models.index(yaml_model)
    cfg = setup_config(yaml_model, wgts_dir)
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, all_wgts[model_index])

    # Step 2: Create a Predictor
    predictor = DefaultPredictor(cfg)
    outputs = predictor(image)

    custom_threshold = threshold
    instances = outputs["instances"]

    # Extract the bounding boxes, class labels, and scores
    boxes = instances.pred_boxes if instances.has("pred_boxes") else None
    scores = instances.scores if instances.has("scores") else None
    classes = instances.pred_classes if instances.has("pred_classes") else None

    # Filter predictions based on the custom threshold
    filtered_indices = scores > custom_threshold
    filtered_boxes = boxes[filtered_indices]
    filtered_scores = scores[filtered_indices]
    filtered_classes = classes[filtered_indices]

    pred_boxes = filtered_boxes.tensor.to('cpu').tolist()
    pred_scores = filtered_scores.to('cpu').tolist()
    pred_classes = filtered_classes.to('cpu').tolist()
    #-------------------------------------------------
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    end_time = time.time()
    execution_time = end_time - start_time
    execution_time = np.round(execution_time,2)

    return pred_boxes, pred_classes, pred_scores, execution_time

In [14]:
model_name = 'yolo'
threshold = 0.1

In [44]:
os.remove('/kaggle/working/Output/DeepOCSORT.mp4')
# Extract ground truth
image = cv2.imread(first_frame)
with open(target_label, 'r') as f:
    lines = f.readlines()

target_gt = lines[0].split()
target_gt = [float(x) for x in target_gt]  # convert string to float
target_bbox = xywhn_to_xyxy(target_gt[1], target_gt[2], target_gt[3], target_gt[4], image_width=image.shape[1], image_height=image.shape[0])
target_img = extract_target_img(image, target_bbox)


tracker = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'), # which ReID model to use
    device=device,
    fp16=fp16
)
vid = cv2.VideoCapture(source)
writer = create_video_writer(vid, out_dir + "DeepOCSORT.mp4")

tracker_predictions = []

frame_no = 0
show = False
debug = []
count = 0
frame_debug_start = 210
frame_debug_end = 979
while True:
    clear_output(wait=False)
    '''if(frame_no >frame_debug_start and frame_no < frame_debug_end):
        show=True
        if (frame_no%20 == 0):
            clear_output(wait=False)
    
    elif(frame_no > frame_debug_end):
        break
    else:
        frame_no +=1
        clear_output(wait=False)'''
        
    ret, frame = vid.read()

  # if video frames end
    if not ret:
        break
    else:
        frame_no += 1
        print('Frame No: ', frame_no)

    if model_name == 'yolo':
        weight = all_wgts[model_names.index(model_name)]
        pred_boxes, pred_classes, pred_scores, execution_time = load_yolo(frame.copy(), weight, threshold)
    else:
        pred_boxes, pred_classes, pred_scores, execution_time = load_detectron2(frame.copy(), model_name, threshold)

        
    filtered_index = filter_bboxes(frame.copy(), pred_boxes, target_img, threshold=0.8, show_filter_img=show, show_scores=show)
    if filtered_index == -1:
        dets = np.empty((0, 6))
    else:
        dets = list(pred_boxes[filtered_index])
        dets.append(pred_scores[filtered_index])
        dets.append(pred_classes[filtered_index])
        dets = np.array([dets])


    ts = tracker.update(dets, frame.copy()) # --> ts output = (x1, y1, x2, y2, id, conf, cls)
    if(ts.shape[0]==0):
        count +=1
        
    if(count>3):
        break

    if show:
        print('Filter Index:', filtered_index)
        print('Detection:', dets)
        print('Tracker Output:', ts)

    xyxys = ts[:,0:4].astype('int') # float64 to int
    ids = ts[:, 4].astype('int') # float64 to int
    confs = np.round(ts[:, 5], 2)
    clss = ts[:, 6]

  # print bboxes with their associated id, cls and conf
    for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
        im = cv2.rectangle(
        frame,
        (xyxy[0], xyxy[1]),
        (xyxy[2], xyxy[3]),
        color,
        thickness
        )
        
        cv2.putText(
        frame,
        f'id: {id}, conf: {conf}, c: {cls}',
        (xyxy[0], xyxy[1]-10),
        cv2.FONT_HERSHEY_SIMPLEX,
        fontscale,
        color,
        thickness
        )

    # save tracker predictions for benchmark
    # <frame_no>, <tracke_obj_id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <conf>, -1 -1 -1
    # <frame_no>, <tracke_obj_id>, <x1>, <y1>, <x2-x1>, <y2-y1>, <conf>, -1 -1 -1
    pred = f"{frame_no}, {id}, {xyxy[0]}, {xyxy[1]}, {xyxy[2]-xyxy[0]}, {xyxy[2]-xyxy[0]}, {conf}, -1, -1, -1"
    tracker_predictions.append(pred)

    debug_txt = f'{frame_no}, Dets:{dets}, Ts:{ts}'
    debug.append(debug_txt)
    if show:
        cv2_imshow(frame, size=(8, 6))
    
    cv2_imshow(frame, show=False, save=f'/kaggle/working/Diagnosis/testFrame_{frame_no}.png')
    
    writer.write(frame)
        


vid.release()
writer.release()

with open(out_dir + "DeepOCSORT_pred.txt", 'w') as mot_file:
    for pred in tracker_predictions:
            mot_file.write(pred + '\n')


print("Task Completed")


Frame No:  506


KeyboardInterrupt: 

In [23]:
debug

['2, Dets:[[     860.19      419.16        1070      753.94     0.63172           1]], Ts:[[     860.19      419.16        1070      753.94           1     0.63172           1]]',
 '4, Dets:[[     859.23      420.68      1071.2      755.82     0.73994           1]], Ts:[[     859.23      420.68      1071.2      755.82           1     0.73994           1]]',
 '6, Dets:[[     860.33      420.84      1071.6      757.55     0.68072           1]], Ts:[[     860.33      420.84      1071.6      757.55           1     0.68072           1]]',
 '8, Dets:[[      862.1      422.83      1071.7      760.31     0.67564           1]], Ts:[[      862.1      422.83      1071.7      760.31           1     0.67564           1]]',
 '10, Dets:[[      858.5      422.39      1075.1      763.26     0.63189           1]], Ts:[[      858.5      422.39      1075.1      763.26           1     0.63189           1]]',
 '12, Dets:[[     860.15       423.7      1075.2      766.08     0.72567           1]], Ts:[[     8

In [ ]:


import shutil

# Define source and destination paths
source_path = '/kaggle/input/files12345/deep_ocsort.py'
destination_path = '/kaggle/working/yolo_tracking/boxmot/trackers/deepocsort/deep_ocsort.py'

os.remove(destination_path)
shutil.copyfile(source_path, destination_path)

# Testing DeepOCSORT

In [ ]:
#help(DeepOCSORT)

In [ ]:
start_time = time.time()
tracemalloc.start()

In [ ]:
tracker = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'), # which ReID model to use
    device=device,
    fp16=fp16
)
vid = cv2.VideoCapture(source)
writer = create_video_writer(vid, out_dir + "DeepOCSORT.mp4")

frameNo=0
while True:       
    clear_output(wait=False)
    ret, im = vid.read()

    # if video frames end
    if not ret:
        break
    else:
        results = model.predict(im)[0]

    if np.array(results.boxes.data.tolist()).ndim < 2:
        continue

    new_results = findTargetObjectIndex(im, results, target_obj_img, frameNo, obj_dir='')
    ts = tracker.update(np.array(new_results.boxes.data.tolist()), im) # --> (x, y, x, y, id, conf, cls)
        
    xyxys = ts[:,0:4].astype('int') # float64 to int
    ids = ts[:, 4].astype('int') # float64 to int
    confs = np.round(ts[:, 5], 2)
    clss = ts[:, 6]

    # print bboxes with their associated id, cls and conf
    if ts.shape[0] != 0:
        for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
            im = cv2.rectangle(
                im,
                (xyxy[0], xyxy[1]),
                (xyxy[2], xyxy[3]),
                color,
                thickness
            )
            cv2.putText(
                im,
                f'id: {id}, conf: {conf}, c: {cls}',
                (xyxy[0], xyxy[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX,
                fontscale,
                color,
                thickness
            )
            # save tracker predictions for benchmark 
            # <frame_no>, <tracke_obj_id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <conf>, -1 -1 -1
            # <frame_no>, <tracke_obj_id>, <x1>, <y1>, <x2-x1>, <y2-y1>, <conf>, -1 -1 -1
            pred = f"{frame_no}, {id}, {xyxy[0]}, {xyxy[1]}, {xyxy[2]-xyxy[0]}, {xyxy[2]-xyxy[0]}, {conf}, -1, -1, -1" 
            tracker_predictions.append(pred)
            frame_no += 1

    # show the frame to our screen

    #writer.write(im)


#vid.release()
#writer.release()

with open(out_dir + "DeepOCSORT_pred.txt", 'w') as mot_file:
    for pred in tracker_predictions:
            mot_file.write(pred + '\n')
            
print("Task Completed")

In [ ]:
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# Calculate the time taken
end_time = time.time()
execution_time = end_time - start_time

Execution_time = np.round(execution_time,2)
Current_memory = np.round(current / 1024 / 1024, 2)
Peak_memory = np.round(peak / 1024 / 1024, 2)

performance_DeepOCSORT = [Execution_time, Current_memory, Peak_memory]

print(f"Execution time: {Execution_time} seconds")
print(f"Current memory usage: {Current_memory} MB")
print(f"Peak memory usage: {Peak_memory} MB")

# Testing StrongSORT


In [ ]:
#help(StrongSORT)

In [ ]:
shutil.copy("/kaggle/input/newfiless/strong_sort.py", "/kaggle/working/yolo_tracking/boxmot/trackers/strongsort/")

In [ ]:
start_time = time.time()
tracemalloc.start()

In [ ]:
tracker = StrongSORT(
    model_weights=Path('mobilenetv2_x1_4_dukemtmcreid.pt'), # which ReID model to use
    device=device,
    fp16=fp16,
)
tracker.n_init = 1
vid = cv2.VideoCapture(source)
writer = create_video_writer(vid, out_dir +"StrongSort.mp4")

tracker_predictions = []

frame_no=1
while True:
    #clear_output(wait=False)
    ret, im = vid.read()

    # if video frames end
    if not ret:
        break
    else:
        #im_resized = cv2.resize(im, (640, 640))
        results = model.predict(im)[0]

    if np.array(results.boxes.data.tolist()).ndim < 2:
        detactions = np.empty((0,6))
    else:
        new_results = findTargetObjectIndex(im, results, target_obj_img, frame_no, obj_dir='')
        if (new_results == -1):
            detactions = np.empty((0,6))
        else:
            detactions = np.array(new_results.boxes.data.tolist())
        
    print(detactions)
    ts = tracker.update(detactions, im) # --> (x, y, x, y, id, conf, cls)
    print(ts)
    if (ts.size == 0):
        ts = np.empty((0,7))
        
    # print bboxes with their associated id, cls and conf
    xyxy = ts[:,0:4].astype('int') # float64 to int
    id = ts[:, 4].astype('int') # float64 to int
    conf = np.around(ts[:, 5])
    cls = ts[:, 6]

    
    im = cv2.rectangle(
        im,
        (xyxy[0], xyxy[1]),
        (xyxy[2], xyxy[3]),
        color,
        thickness
    )
    cv2.putText(
        im,
        f'id: {id}, conf: {conf}, c: {cls}',
        (xyxy[0], xyxy[1]-10),
        cv2.FONT_HERSHEY_SIMPLEX,
        fontscale,
        color,
        thickness
    )
    # save tracker predictions for benchmark 
    # <frame_no>, <tracke_obj_id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <conf>, -1 -1 -1
    # <frame_no>, <tracke_obj_id>, <x1>, <y1>, <x2-x1>, <y2-y1>, <conf>, -1 -1 -1
    pred = f"{frame_no}, {id}, {xyxy[0]}, {xyxy[1]}, {xyxy[2]-xyxy[0]}, {xyxy[2]-xyxy[0]}, {conf}, -1, -1, -1" 
    tracker_predictions.append(pred)
    frame_no += 1

    # show the frame to our screen

    #writer.write(im)

#vid.release()
#writer.release()

with open("/kaggle/working/Test_pred.txt", 'w') as mot_file:
    for pred in tracker_predictions:
            mot_file.write(pred + '\n')


print("Task Completed")
print(frame_no)

In [ ]:
ts

In [ ]:
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# Calculate the time taken
end_time = time.time()
execution_time = end_time - start_time

Execution_time = np.round(execution_time,2)
Current_memory = np.round(current / 1024 / 1024, 2)
Peak_memory = np.round(peak / 1024 / 1024, 2)
performance_StrongSORT = [Execution_time, Current_memory, Peak_memory]

print(f"Execution time: {Execution_time} seconds")
print(f"Current memory usage: {Current_memory} MB")
print(f"Peak memory usage: {Peak_memory} MB")

## Testing BoTSORT

In [ ]:
#help(BoTSORT)

In [ ]:
start_time = time.time()
tracemalloc.start()

In [ ]:
from boxmot import BoTSORT
tracker = BoTSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'),
    device=device,
    fp16=fp16,
)
vid = cv2.VideoCapture(source)
writer = create_video_writer(vid, out_dir + "BoTSORT.mp4")

while True:
    clear_output(wait=False)
    ret, im = vid.read()

    # if video frames end
    if not ret:
        break
    else:
        #im_resized = cv2.resize(im, (640, 640))
        results = model.predict(im)[0]

    if np.array(results.boxes.data.tolist()).ndim < 2:
        continue

    new_results = findTargetObjectIndex(im, results, target_obj_img, frameNo, obj_dir='')
    try:
        ts = tracker.update(np.array(new_results.boxes.data.tolist()), im) # --> (x, y, x, y, id, conf, cls)
        if (ts.size == 0):
            ts = np.array( [[0,0,0,0,0,0,0]] )
    except:
        ts = np.array( [[0,0,0,0,0,0,0]] )
        

    xyxys = ts[:,0:4].astype('int') # float64 to int
    ids = ts[:, 4].astype('int') # float64 to int 
    confs = np.round(ts[:, 5])
    clss = ts[:, 6]

    # print bboxes with their associated id, cls and conf
    if ts.shape[0] != 0:
        for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
            im = cv2.rectangle(
                im,
                (xyxy[0], xyxy[1]),
                (xyxy[2], xyxy[3]),
                color,
                thickness
            )
            cv2.putText(
                im,
                f'id: {id}, conf: {conf}, c: {cls}',
                (xyxy[0], xyxy[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX,
                fontscale,
                color,
                thickness
            )

    # show the frame to our screen
    
    writer.write(im)


vid.release()
writer.release()
print("Task Completed")

In [ ]:
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# Calculate the time taken
end_time = time.time()
execution_time = end_time - start_time

Execution_time = np.round(execution_time,2)
Current_memory = np.round(current / 1024 / 1024, 2)
Peak_memory = np.round(peak / 1024 / 1024, 2)

performance_BoTSORT = [Execution_time, Current_memory, Peak_memory]

print(f"Execution time: {Execution_time} seconds")
print(f"Current memory usage: {Current_memory} MB")
print(f"Peak memory usage: {Peak_memory} MB")

## Testing BYTETracker

In [ ]:
#help(BYTETracker)

In [ ]:
start_time = time.time()
tracemalloc.start()

In [ ]:
from boxmot import BYTETracker
tracker =  BYTETracker()
vid = cv2.VideoCapture(source)
writer = create_video_writer(vid, out_dir+"BYTETracker.mp4")

while True:
    clear_output(wait=False)
    ret, im = vid.read()

    # if video frames end
    if not ret:
        break
    else:
        #im_resized = cv2.resize(im, (640, 640))
        results = model.predict(im)[0]

    if np.array(results.boxes.data.tolist()).ndim < 2:
        continue

    new_results = findTargetObjectIndex(im, results, target_obj_img, frameNo, obj_dir='')
    try:
        ts = tracker.update(np.array(new_results.boxes.data.tolist()), im) # --> (x, y, x, y, id, conf, cls)
        if (ts.size == 0):
            ts = np.array( [[0,0,0,0,0,0,0]] )
    except:
        ts = np.array( [[0,0,0,0,0,0,0]] )
        
        
    xyxys = ts[:,0:4].astype('int') # float64 to int
    ids = ts[:, 4].astype('int') # float64 to int
    confs = np.round(ts[:, 5])
    clss = ts[:, 6]

    # print bboxes with their associated id, cls and conf
    if ts.shape[0] != 0:
        for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
            im = cv2.rectangle(
                im,
                (xyxy[0], xyxy[1]),
                (xyxy[2], xyxy[3]),
                color,
                thickness
            )
            cv2.putText(
                im,
                f'id: {id}, conf: {conf}, c: {cls}',
                (xyxy[0], xyxy[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX,
                fontscale,
                color,
                thickness
            )

    # show the frame to our screen
    
    writer.write(im)


vid.release()
writer.release()
print("Task Completed")

In [ ]:
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# Calculate the time taken
end_time = time.time()
execution_time = end_time - start_time

Execution_time = np.round(execution_time,2)
Current_memory = np.round(current / 1024 / 1024, 2)
Peak_memory = np.round(peak / 1024 / 1024, 2)

performance_BYTETracker = [Execution_time, Current_memory, Peak_memory]

print(f"Execution time: {Execution_time} seconds")
print(f"Current memory usage: {Current_memory} MB")
print(f"Peak memory usage: {Peak_memory} MB")

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(list(zip(performance_DeepOCSORT, performance_StrongSORT, performance_BoTSORT, performance_BYTETracker)),
                  columns = ['DeepOCSORT', 'StrongSORT', 'BoTSORT', 'BYTETracker'],
                  index = ['Execution_time(s)', 'Current_memory(MB)', 'Peak_memory(MB)'])

df.to_csv(out_dir + 'SimulationPerformance.csv')

In [ ]:
import os
import shutil

for i in [4,5,6]:
    OUTPUT_NAME = f'/kaggle/working/TrackResultsTest{i}'
    DIRECTORY_TO_ZIP = f'/kaggle/working/Test{i}'
    
    print(DIRECTORY_TO_ZIP)
    shutil.make_archive(OUTPUT_NAME, 'zip', DIRECTORY_TO_ZIP)

In [ ]:
import os
import shutil
#for file in os.listdir('/kaggle/working/Diagnosis'):
    #os.remove('/kaggle/working/Diagnosis/' + file)
    
shutil.rmtree('/kaggle/working/yolo_tracking')